# LangChain を使用した Bedrock によるコンテキストを含むプロンプト

> *このノートブックは、SageMaker Studioの **`Data Science 3.0`** カーネルで実行してください*

## 概要

このノートブックでは、顧客サポートエンジニアから受けたカスタマーサービスの品質に満足していない顧客に対するEメール応答を生成する方法を示します。 不満を抱える顧客から実際に受信したEメールの内容をモデルに提供することで、追加のコンテキストを提供します。

プロンプトに追加のコンテキストがあるため、このノートブックの Amazon Titan モデルによって生成されたテキストは、ゼロショットプロンプトを通じて以前に生成されたコンテンツよりもるかに高品質で関連性が高いものになっています。

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) は、言語モデルを使用したアプリケーションを開発するためのフレームワークです。このフレームワークの主な側面は、さまざまなコンポーネントを連鎖させることで大規模言語モデルを拡張し、高度なユースケースを作成できるようにすることです。

**注:** *このノートブックは、AWS環境の内外で実行できます。*

#### コンテキスト
前の例 `Lab1a_jp.ipynb` では、Boto3 SDK を使ってAmazon Bedrock API を実行する方法を確認しました。このノートブックでは、LangChainフレームワークのPromptTemplatesを使って、少しだけ複雑さを追加しようと思います。PromptTemplates を使うと、後で情報を埋め込むことができる汎用のテンプレートを作成し、さまざまなシナリオに基づいてモデルの出力を取得することができます。

このノートブックの一環として、LangChain フレームワーク内の Amazon Bedrock の統合方法と、PromptTemplate の助けを借りてテキストを生成する方法を探究します。

#### パターン
ここでは、 Amazon Bedrock API と LangChaint を使って、追加の例を提供することなく、タスク、指示、モデルへの入力から成る単純な入力から、モデルの出力を取得する方法を確認します。 ここでの目的は、強力な大規模言語モデルが与えられたタスクを理解し、説得力のある出力を生成できることを示すことです。

![](./images/chatbot_bedrock.png)

#### ユースケース
Amazon Bedrockのモデルの生成能力を示すために、Eメール生成のユースケースを取り上げます。

#### ペルソナ
あなたは AnyCompany 社のカスタマーサービスマネージャーのボブです。あなたの顧客の中には、カスタマーサポートエンジニアによって提供されたサービスに満足していない人がおり、ネガティブなフィードバックをしています。あなたは顧客に低品質なサービスについて謝罪し、信頼を回復したいと考えています。わかりやすく、かつメールを送った顧客にパーソナライズされたメールを大量作成するのに、LLM の助けが必要です。

#### 実装
このユースケースを満たすために、顧客の以前のメールに基づいて感謝のメールを生成する方法を示します。 Amazon Bedrock LangChain 統合を使用して、Amazon Titan モデルを使用します。

In [ ]:
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from labutils import bedrock, print_ww


# ---- ⚠️ AWS 環境の設定に応じて、以下の行のコメントを外して編集してください。 ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Bedrock LLM を実行する

まず、llms から Bedrock クラスのインスタンスを作成します。ここでは、Amazon Bedrock で利用できるモデルの ARN である model_id を指定します。

オプションで、以前に作成したboto3クライアントや、`temperature`、`topP`、`maxTokenCount`、`stopSequences` などのパラメータを保持できる model_kwargs を渡すこともできます(パラメータの詳細はAmazon Bedrockコンソールで確認できます)。

Amazon Bedrock で利用できるテキスト生成モデルの ID は [documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids-arns.html) を確認してください。

モデル毎に異なる model_kwargs をサポートしていることに注意してください。

In [ ]:
from langchain_community.llms.bedrock import Bedrock
# from langchain.llms.bedrock import Bedrock 

inference_modifier = {
    'max_tokens_to_sample':4096, 
    "temperature":0.5,
    "top_k":250,
    "top_p":1,
    "stop_sequences": ["\n\nHuman"]
}

textgen_llm = Bedrock(model_id = "anthropic.claude-v2:1", # 使用する環境に合わせて、適切なモデルバージョンを指定してください。
    client = boto3_bedrock, 
    model_kwargs = inference_modifier 
)


## LangChain カスタムプロンプトの作成

プロンプトのテンプレートを作成することで、実行ごとに異なる入力変数を渡すことができます。これは、データベースから取得する可能性のある異なる入力変数でコンテンツを生成する必要がある場合などに有効です。

前のラボではプロンプトをハードコーディングしていましたが、同様の否定的なフィードバックを送信する顧客が複数いて、それぞれの顧客に謝罪の応答をする際に、顧客のメールに応じてメールの内容をある程度パーソナライズしたいと考えています。次のセルでは、このパターンを実現するために `PromptTemplate` を作成する方法を説明します。

In [ ]:
from langchain.prompts import PromptTemplate

# 複数の入力変数を持つプロンプトテンプレートを作成
multi_var_prompt = PromptTemplate(
    input_variables=["customerServiceManager", "customerName", "feedbackFromCustomer"], 
    template="""

Human: Create an apology email from the Service Manager {customerServiceManager} to {customerName} in response to the following feedback that was received from the customer: 
<customer_feedback>
{feedbackFromCustomer}
</customer_feedback>

Assistant:"""
)

# 入力変数に値を設定
prompt = multi_var_prompt.format(customerServiceManager="Bob", 
                                 customerName="John Doe", 
                                 feedbackFromCustomer="""Hello Bob,
     I am very disappointed with the recent experience I had when I called your customer support.
     I was expecting an immediate call back but it took three days for us to get a call back.
     The first suggestion to fix the problem was incorrect. Ultimately the problem was fixed after three days.
     We are very unhappy with the response provided and may consider taking our business elsewhere.
     """
     )


In [ ]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

## 再実行

プロンプトテンプレートを使ってモデルを実行し、応答を確認します。

In [ ]:
response = textgen_llm.invoke(prompt)

email = response[response.index('\n')+1:]

print_ww(email)

## まとめ

コンテキストなしで LLM を呼び出すと、望ましい結果が得られない可能性があることが分かりました。  
コンテキストを追加し、プロンプトテンプレートを使用してLLMからの出力を制約することで、望ましい出力を得ることができました。